#### Installation

In [1]:
#!pip install git+https://www.github.com/huggingface/transformers
#!pip install git+https://github.com/huggingface/accelerate
#!pip install bitsandbytes
#!pip install einops
#!pip install --upgrade torch torchvision
#!pip install scikit-learn
#!pip install matplotlib
#!pip install datasets
#!pip install Bio
#!pip install pybedtools
#!pip install tabulate


### Libraries (some in utility_fuction.py)

In [2]:
# Hugging Face Datasets library
from datasets import load_dataset, Dataset

# Scikit-learn for evaluation and data splitting
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Biopython for sequence processing
from Bio import SeqIO

# PyBedTools for genomic interval manipulation
from pybedtools import BedTool

# Data analysis and scientific computation
import pandas as pd
import numpy as np
import scipy.stats as stats

# PyTorch for deep learning
import torch
from torch import nn

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities and external libraries
import os
import pickle
import genopyc
import utility_functions as uf # Custom utility functions for loading models/tokenizers, datasets from the experiments

from concurrent.futures import ThreadPoolExecutor
import time

# Configure Seaborn for better plotting aesthetics
sns.set()

2025-01-06 06:02:01.215893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-06 06:02:01.215970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-06 06:02:01.347385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 06:02:01.642304: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-06 06:02:06.005274: W tensorflow/compiler/tf2

### Parameters (load models for prediction)

In [3]:
# model name from huggingface.co/model name_id:model_name
models_names = [
 'dnabert2',
 'nucleotide-transformer-v2-50m-multi-species',
 'nucleotide-transformer-v2-100m-multi-species',
 'nucleotide-transformer-v2-250m-multi-species',
 'nucleotide-transformer-v2-500m-multi-species',
 'nucleotide-transformer-500m-1000g',
 'nucleotide-transformer-500m-human-ref',
 'nucleotide-transformer-2.5b-1000g',
 'nucleotide-transformer-2.5b-multi-species',
 'Geneformer',
 'gena-lm-bert-base-t2t',
 'gena-lm-bert-large-t2t',
 'gena-lm-bert-base-t2t-multi',
 'gena-lm-bigbird-base-t2t',
 'hyenadna-small-32k-seqlen-hf',
 'hyenadna-medium-160k-seqlen-hf',
 'hyenadna-medium-450k-seqlen-hf',
 'hyenadna-large-1m-seqlen-hf'
 ]

# type of fine-tuned
ft_model_type = '1kbpHG19_DHSs_H3K27AC'

# samples for fine-tuning
#'BioS2'=Hela, 'BioS45'=neural progenitor cell, 'BioS73'=hepg2, 'BioS74'=k562
bios_ids = ['BioS2', 'BioS45', 'BioS73', 'BioS74']

# load chrom2seq hg38 and hg19
with open(f'/data/Dcode/gaetano/repos/AI4Genomic/data/chrom2seqHG38.pkl', "rb") as pickle_file:
    chrom2seqHG38 = pickle.load(pickle_file)    

with open(f'/data/Dcode/gaetano/repos/AI4Genomic/data/chrom2seqHG19.pkl', "rb") as pickle_file:
    chrom2seqHG19 = pickle.load(pickle_file)

## Main

#### Load experiment dataset

In [4]:
data_to_source = pd.read_csv('/data/Dcode/gaetano/repos/AI4Genomic/data/data_mutagenesis_curated.csv')
data_to_source

,name,path,hg type,#samples,cell line,study,resource,BioS,type_data
0,k562.sign.id.LP190708.txt,/data/Dcode/gaetano/repos/AI4Genomic/data/raQT...,hg19,19237,k562,PMID: 31253979,https://osf.io/6y9td/,BioS74,raQTL
1,hepg2.sign.id.LP190708.txt,/data/Dcode/gaetano/repos/AI4Genomic/data/raQT...,hg19,14183,hepg2,PMID: 31253979,https://osf.io/6y9td/,BioS73,raQTL
2,SORT1.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg38,1789,hepg2,PMID: 31395865,https://pavlopoulos-lab.org/shinyapps/app/mpra...,BioS73,mpra
3,GSE87711.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg19,84,k562,PMID: 27259154,paper Sup. Mat.,BioS74,mpra
4,GSE68331_SNP.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg19,283,hepg2,PMID: 26084464,paper Sup. Mat.,BioS73,mpra
5,NPC_SNP.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg19,14042,NPC,PMID: 33885362,paper Sup. Mat.,BioS45,mpra
6,Hela_TRAF3IP2.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg19,1962,Hela,PMID: 25340400,paper Sup. Mat.,BioS2,mpra
7,Hela_PPARG.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg19,1614,Hela,PMID: 25340400,paper Sup. Mat.,BioS2,mpra
8,Hela_SORL1.csv,/data/Dcode/gaetano/repos/AI4Genomic/data/mpra...,hg19,1665,Hela,PMID: 25340400,paper Sup. Mat.,BioS2,mpra


### preprocess experimental dataset

In [5]:
idx = 0
data_ = data_to_source.iloc[idx]

# extract from dataset
dataset_df = uf.data_preprocessing_experimental_result(data_['type_data'], data_['name'], data_['path'])
print(data_['type_data'], data_['name'])
dataset_df

raQTL k562.sign.id.LP190708.txt


,Chromosome,Position,Reference,Alternative,SNP_id,Value_Ratio,Value_Diff,Value_Pvalue_signed,P_value
0,chr5,1559341,T,C,rs10056572,22.284285,10.881131,32.692414,2.030421e-33
1,chr18,68919785,C,G,rs991512,0.052253,-56.034510,-32.141415,7.220797e-33
2,chr14,89393045,G,A,rs7156067,0.112420,-7.943033,-30.536830,2.905157e-31
3,chr4,59436721,T,C,rs1605769,15.647984,14.122534,30.526140,2.977554e-31
4,chr14,72218104,A,G,rs1204985,0.059336,-22.123020,-28.740874,1.816043e-29
...,...,...,...,...,...,...,...,...,...
19232,chr6,141445279,G,C,rs1488530,0.729299,-1.963231,-2.208381,6.188983e-03
19233,chr6,164613457,C,T,rs1034447,0.509704,-3.379048,-2.208293,6.190229e-03
19234,chr8,39801184,G,A,rs6983467,2.539078,9.126143,2.208282,6.190392e-03
19235,chr1,81050437,C,A,rs7535894,1.563135,1.754667,2.208235,6.191056e-03


### get LD block for each SNP (R>0.8)

In [6]:
def get_association_alleles(SNP_ids, rate_limit=15):
    def fetch_variants(snp_id):
        return snp_id, genopyc.get_variants_in_LD(snp_id, 0.8)

    alleles_dict = {}
    batch_size = rate_limit  # Process up to `rate_limit` SNPs at a time
    for i in range(0, len(SNP_ids), batch_size):
        batch = SNP_ids[i:i+batch_size]
        with ThreadPoolExecutor() as executor:
            results = executor.map(fetch_variants, batch)
            alleles_dict.update(results)

    return alleles_dict

# Example usage
result = get_association_alleles(dataset_df['SNP_id'].tolist())

## remove NoneType keys from the dictionary
result = {k: v for k, v in result.items() if v is not None}

## remove keys where the list is empty 
result = {k: v for k, v in result.items() if v}  

result.keys()

ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388066072."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388066073."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388102049."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388038198."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388078240."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388056631."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388074547."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388078237."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: rs4073650."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388046876."}
ERROR: Bad Request:
{"error":"Could not fetch variation object for id: ss1388073770."}
ERROR: Bad Request:
{"error":"Could not fetch 

dict_keys(['rs10056572', 'rs991512', 'rs7156067', 'rs1605769', 'rs1204985', 'rs117788163', 'rs5758951', 'rs7495131', 'rs12302520', 'rs8017153', 'rs1957777', 'rs73551757', 'rs1858929', 'rs6732307', 'rs332669', 'rs12463108', 'rs11737139', 'rs1639945', 'rs148668002', 'rs182822', 'rs1916027', 'rs59759655', 'rs7784142', 'rs565902955', 'rs4814799', 'rs4923818', 'rs73653456', 'rs78995987', 'rs12899345', 'rs443623', 'rs60146234', 'rs2349075', 'rs1377374', 'rs16912908', 'rs1399241', 'rs28550024', 'rs12925824', 'rs73551756', 'rs2330578', 'rs12827804', 'rs1851307', 'rs150080959', 'rs7872857', 'rs10409076', 'rs10415333', 'rs12632163', 'rs10938319', 'rs2153531', 'rs3852061', 'rs62466605', 'rs73073006', 'rs1475740', 'rs10773052', 'rs880033', 'rs7329291', 'rs12939076', 'rs400824', 'rs1781800', 'rs7115896', 'rs6657320', 'rs7303735', 'rs12662316', 'rs17235982', 'rs2263342', 'rs2793674', 'rs10904052', 'rs62451633', 'rs636494', 'rs3748136', 'rs7116006', 'rs1841941', 'rs3864109', 'rs6830311', 'rs2032312',

#### save dict LD

In [7]:
## save result as pickle file 
with open(f"/data/Dcode/gaetano/repos/AI4Genomic/data/ld/ld_{data_['name']}.pkl", "wb") as pickle_file:
    pickle.dump(result, pickle_file)

print(f"Done - ld {data_['name']} saved")

Done - ld k562.sign.id.LP190708.txt saved


#### open dict of LDs  

In [6]:
## open pickle 
with open(f"/data/Dcode/gaetano/repos/AI4Genomic/data/ld/ld_{data_['name']}.pkl", "rb") as pickle_file:
    result = pickle.load(pickle_file)

print(f"Loaded LD dictionary for {data_['name']}")

Loaded LD dictionary for k562.sign.id.LP190708.txt


In [7]:
len(result.keys())

14824

### create a datafram from LD blocks

In [10]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to process a single SNP candidate
def process_snp_candidate(snp_candidate):
    local_data = []
    try:
        # Fetch variant information
        list_snps = result[snp_candidate]
        info_ld = genopyc.get_variants_info(list_snps, chunked=False, chunksize=200)
        
        # Extract and filter relevant data
        for snp_id, snp_info in info_ld.items():
            mappings = snp_info.get('mappings', [])
            if not mappings:
                continue

            allele_string = mappings[0].get('allele_string', '')
            location = mappings[0].get('location', '')
            if not allele_string or not location:
                continue

            # Parse alleles and position
            alleles = allele_string.split('/')
            pos = location.split(':')
            if len(pos) != 2 or len(alleles) != 2:
                continue

            chromosome = f"chr{pos[0]}"
            position = int(pos[1].split('-')[0])
            reference, alternative = alleles
            if reference == '-' or alternative=='-':
                continue

            # Include only SNPs (single nucleotide changes)
            if len(reference) == 1 and len(alternative) == 1:
                local_data.append([chromosome, position, reference, alternative, snp_id, snp_candidate])
    except Exception as e:
        print(f"Error processing {snp_candidate}: {e}")
    return local_data


# Process keys in batches of 15
data = []
keys = list(result.keys())

batch_size = 15
with ThreadPoolExecutor(max_workers=batch_size) as executor:
    for i in tqdm(range(0, len(keys), batch_size), desc="Processing SNP Batches"):
        batch = keys[i:i + batch_size]
        future_to_key = {executor.submit(process_snp_candidate, snp): snp for snp in batch}
        
        for future in as_completed(future_to_key):
            result_data = future.result()
            if result_data:
                data.extend(result_data)

# Convert collected data into a DataFrame
df = pd.DataFrame(data, columns=['Chromosome', 'Position', 'Reference', 'Alternative', 'SNP_id', 'SNP_id_candidate'])
df


Processing SNP Batches:   0%|          | 4/989 [01:37<7:28:37, 27.33s/it]

total number of chunks: 2
chunk 0 processed
chunk 1 processed


Processing SNP Batches:   1%|          | 5/989 [01:58<6:52:29, 25.15s/it]

total number of chunks: 2
chunk 0 processed


Processing SNP Batches:   1%|          | 6/989 [02:20<6:32:39, 23.97s/it]

...

chunk 1 processed
total number of chunks: 2
chunk 0 processed


Processing SNP Batches: 100%|█████████▉| 986/989 [9:01:28<02:19, 46.39s/it]

chunk 1 processed


Processing SNP Batches: 100%|██████████| 989/989 [9:02:18<00:00, 32.90s/it]


,Chromosome,Position,Reference,Alternative,SNP_id,SNP_id_candidate
0,chr14,63130826,T,C,rs2883863,rs1957777
1,chr14,63102579,A,G,rs11158473,rs1957777
2,chr9,130397274,G,A,rs73551756,rs73551757
3,chr19,46242477,A,G,rs150221996,rs117788163
4,chr14,88928553,C,T,rs61982566,rs7156067
...,...,...,...,...,...,...
316875,chr6,141045765,C,T,rs1856374,rs1488530
316876,chr6,140991230,G,A,rs1819633,rs1488530
316877,chr6,141066950,C,T,rs2788372,rs1488530
316878,chr6,141066002,T,C,rs11155161,rs1488530


#### Save LD dataframe

In [11]:
## save dataframe as pickle file 
with open(f"/data/Dcode/gaetano/repos/AI4Genomic/data/ld/ld_df_{data_['name']}.pkl", "wb") as pickle_file:
    pickle.dump(df, pickle_file)

print(f"Done - ld df {data_['name']} saved")

Done - ld df k562.sign.id.LP190708.txt saved


#### Open LD dataframe

In [8]:
## load dataframe 
with open(f"/data/Dcode/gaetano/repos/AI4Genomic/data/ld/ld_df_{data_['name']}.pkl", "rb") as pickle_file:
    df = pickle.load(pickle_file)

print(f"Loaded LD dataframe for {data_['name']}")

Loaded LD dataframe for k562.sign.id.LP190708.txt


### Get Preditions

### Load models and tokenizers

In [9]:
models_tokenizers_dict = uf.load_models_and_tokenizers(models_names, data_['BioS'], ft_model_type)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading model and tokenizer for: tanoManzo/dnabert2_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-v2-50m-multi-species_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-v2-100m-multi-species_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-v2-250m-multi-species_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-v2-500m-multi-species_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-500m-1000g_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-500m-human-ref_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/nucleotide-transformer-2.5b-1000g_ft_BioS74_1kbpHG19_DHSs_H3K27AC


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading model and tokenizer for: tanoManzo/nucleotide-transformer-2.5b-multi-species_ft_BioS74_1kbpHG19_DHSs_H3K27AC


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

Loading model and tokenizer for: tanoManzo/gena-lm-bert-large-t2t_ft_BioS74_1kbpHG19_DHSs_H3K27AC


Some weights of BertForMaskedLM were not initialized from the model checkpoint at tanoManzo/gena-lm-bert-large-t2t_ft_BioS74_1kbpHG19_DHSs_H3K27AC and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model and tokenizer for: tanoManzo/gena-lm-bert-base-t2t-multi_ft_BioS74_1kbpHG19_DHSs_H3K27AC


Some weights of BertForMaskedLM were not initialized from the model checkpoint at tanoManzo/gena-lm-bert-base-t2t-multi_ft_BioS74_1kbpHG19_DHSs_H3K27AC and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model and tokenizer for: tanoManzo/gena-lm-bigbird-base-t2t_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/hyenadna-small-32k-seqlen-hf_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/hyenadna-medium-160k-seqlen-hf_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/hyenadna-medium-450k-seqlen-hf_ft_BioS74_1kbpHG19_DHSs_H3K27AC
Loading model and tokenizer for: tanoManzo/hyenadna-large-1m-seqlen-hf_ft_BioS74_1kbpHG19_DHSs_H3K27AC


### prediction

In [10]:
condition = (df['Chromosome']!='chrHSCHR22_1_CTG7') & (df['Chromosome']!='chrHSCHR11_3_CTG1') & (df['Chromosome']!='chrHG1651_PATCH')

df_filtered = df[condition]
df_filtered.Chromosome.value_counts()

Chromosome
chr1     23684
chr3     23487
chr2     23044
chr4     22106
chr6     21163
chr5     19186
chr11    18195
chr10    17987
chr8     17877
chr7     16889
chr12    15635
chrX     13803
chr9     12516
chr14     9582
chr15     9535
chr13     9259
chr20     7881
chr19     7756
chr18     7286
chr17     6835
chr16     6687
chr22     3468
chr21     3019
Name: count, dtype: int64

In [11]:
# Extract sequences
data_df = uf.extract_SNPsequences_from_df(df_filtered, chrom2seqHG38)

# Display the DataFrame
models_predictions = uf.get_predictions_raw(models_tokenizers_dict, data_df['Seq_Reference'].to_list(), data_df['Seq_Alternative'].to_list() , batch_size=8)

  
# Save dictionary to a pickle file
with open(f"/data/Dcode/gaetano/repos/AI4Genomic/data/predictions/tmp_ld_predictions_{data_['name']}.pkl", "wb") as pickle_file:    
    pickle.dump(models_predictions, pickle_file)

print(f"Done - ld prediction {data_['name']} saved")

Processing model: dnabert2_ft_BioS74
Processing model: nucleotide-transformer-v2-50m-multi-species_ft_BioS74
Processing model: nucleotide-transformer-v2-100m-multi-species_ft_BioS74
Processing model: nucleotide-transformer-v2-250m-multi-species_ft_BioS74
Processing model: nucleotide-transformer-v2-500m-multi-species_ft_BioS74
Processing model: nucleotide-transformer-500m-1000g_ft_BioS74
Processing model: nucleotide-transformer-500m-human-ref_ft_BioS74
Processing model: nucleotide-transformer-2.5b-1000g_ft_BioS74
Processing model: nucleotide-transformer-2.5b-multi-species_ft_BioS74
